In [1]:
# Establish connections to the input db and variables spreadsheets

# Import modules
import os, re, math
import arcpy
import pandas as pd
import numpy as np
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from dotenv import load_dotenv

# Lot the .env variables
load_dotenv()

# Set the data paths
inputs_path = os.getenv('J111_INPUTS')
data_path = os.getenv('J122_DATA')
spatial_path = os.getenv('J111_SPATIAL')
env_path = os.getenv('J111_ENV')

# Set path to water consumption data
wc_data_path = os.path.join(data_path, 'supplied/20220209_redland_water')

In [2]:
''' 
----------------------------------
Import water consumption data
----------------------------------
'''
# Get path to 2021 Redland Water consumption data
wc_xlsx = os.path.join(wc_data_path, '2021_redlandwater_consumption_data.xlsx')

# Import development assessment data table
wc_data = pd.read_excel(wc_xlsx, 'DATA')

# Select only the necessary columns
wc_data = wc_data.iloc[:, 0:7]

# Inspect
wc_data.head()

,Meter ID,Property ID,Consumption,AvgConsumption,ServiceType,Tariff,Billing Cycle
0,17W069552,9164,3.000000,0.034483,Potable,WC02,20/21Q4
1,17W069552,9164,7.500000,0.083333,Potable,WC02,21/22Q1
2,17W069552,9164,14.000000,0.152174,Potable,WC02,21/22Q2
3,17W069552,9164,19.500000,0.214286,Potable,WC02,21/22Q3
4,12E002725,227820,130.553094,1.500610,Potable,WC02,20/21Q4


In [3]:
''' 
----------------------------------
Get annual consumption by Property ID
----------------------------------
'''

ann_cons = wc_data.iloc[:, 1:3].groupby(['Property ID']).agg({'Consumption' : 'sum'}).reset_index()

ann_cons = ann_cons.rename({'Property ID' : 'propno', 'Consumption' : 'consumption'}, axis=1)

ann_cons.head()

,propno,consumption
0,1,126.0
1,3,164.0
2,4,148.0
3,5,212.0
4,6,62.0


In [25]:
''' 
----------------------------------
Merge consumption data onto Council current land table
----------------------------------
'''

# Set current land input path
curr_gdb_path = os.path.join(spatial_path, 'redland_gam_current_dev.gdb')

# Import the Council Property features into dataframe
land = pd.DataFrame.spatial.from_featureclass(os.path.join(curr_gdb_path, 'INPUT_FC_CouncilProperty')).drop(['OBJECTID', 'SHAPE'], axis=1)

# Keep only the necessary fields
land = land.loc[:, ('landno', 'propno', 'prop_use', 'zone')]

# Import property use / development groups table from input
missing_propuse = pd.read_excel(os.path.join(inputs_path, 'Redland_GAM_Input_Vars.xlsx'), 'MissingPropertyUse')

# Set the missing propuse on input table
for index, row in missing_propuse.iterrows():
    land.loc[land['landno'] == row.landno, 'prop_use'] = row.prop_use

# Use outerjoin to merge data
land = pd.merge(land, ann_cons, on='propno', how='left')

# Fill and missing values as 0
land.consumption = land.consumption.fillna(0.0)

# Inspect dataframe
land.head()

,landno,propno,prop_use,zone,consumption
0,998033,432430,RES001,LDR,4.0
1,998037,432470,RES001,LDR,31.0
2,998036,432460,RES001,LDR,227.0
3,998038,432480,RES001,LDR,194.0
4,998039,432490,RES001,LDR,104.0


In [5]:
''' 
----------------------------------
Merge in gfa data for each land parcel
----------------------------------
'''

# Import the Building Footprints data
bldg = pd.DataFrame.spatial.from_featureclass(os.path.join(curr_gdb_path, 'INPUT_FC_CouncilProperty_BldgFootprints')).drop(['OBJECTID', 'SHAPE'], axis=1)

# Calculate GFA for each footprint
bldg['gfa_est'] = bldg['bldg_ftprnt_area'] * bldg['bldg_floors_est']

# Groupby landno and sum gfa for each land parcel
bldg = bldg.loc[:, ('landno', 'gfa_est')].groupby(by='landno').sum().reset_index()

# Merge gfa to land table
land_bldg = pd.merge(land, bldg, on='landno', how='left')

# Set no gfa to be 0
land_bldg['gfa_est'] = land_bldg['gfa_est'].fillna(0)

# Inspect
land_bldg.head()

,landno,propno,prop_use,zone,consumption,gfa_est
0,998033,432430,RES001,LDR,4.0,403.57
1,998037,432470,RES001,LDR,31.0,50.71
2,998036,432460,RES001,LDR,227.0,246.32
3,998038,432480,RES001,LDR,194.0,449.07
4,998039,432490,RES001,LDR,104.0,35.54


In [6]:
''' 
----------------------------------
Join the land data to the DCDB reference table
----------------------------------
'''
# Import the Council Property to DCDB Base lots relational list
parcels_to_land = pd.DataFrame.spatial.from_table(os.path.join(curr_gdb_path, 'INPUT_TB_DCDB_CouncilPropertyRel')).drop(['OBJECTID'], axis=1)

# Select only the required column
parcels_land = parcels_to_land.loc[:, ('segpar', 'relationships', 'landnos')].rename({'relationships' : 'rels'}, axis=1)

# Split the land numbers string into list
parcels_land.landnos = parcels_land.landnos.apply(lambda v: [int(x) for x in v.split(';')])

# Explode on the land numbers column (this will create duplicate parcel segpar records)
parcels_land = parcels_land.explode('landnos', ignore_index=True).rename({'landnos' : 'landno'}, axis=1)

# Merge in land data
parcels_land = pd.merge(parcels_land.loc[:, ('landno', 'segpar')], land_bldg, on='landno', how='left')

# Inspect
parcels_land.head()

,landno,segpar,propno,prop_use,zone,consumption,gfa_est
0,146617,28663092,53815,RES001,LDR,236.0,326.72
1,996564,28460061,416730,RES014,LDR,0.0,0.02
2,145846,32043083,13401,RES001,CR,82.0,227.08
3,144907,20484023,13844,VAC003,CN,0.0,0.00
4,112272,20689095,50686,RES001,LDR,111.0,306.54


In [7]:
''' 
----------------------------------
Incorporate development measures
----------------------------------
'''

# Set path to the input variables excel workbook
vars_xlsx = os.path.join(inputs_path, 'Redland_GAM_Input_Vars.xlsx')

# Import property use / development groups table from input
devuse = pd.read_excel(vars_xlsx, 'PropertyUse_to_DevGroup')

# Join details to parcels table
parcels_devuse = pd.merge(parcels_land, devuse, on='prop_use', how='left')

# Define non residential columns
dev_cols = {
    'Detached Dwelling' : 'det_dwl',
    'Multiple Dwelling' : 'att_dwl',
    'Accommodation - Long-Term' : 'accom_long',
    'Accommodation - Short-Term' : 'accom_short',
    'Emergency Services' : 'emer_serv',
    'Places of Assembly' : 'pl_of_ass',
    'Childcare' : 'childcare',
    'Education' : 'education',
    'Care Accommodation' : 'care_accom',
    'Hospital' : 'hospital',
    'Health Services' : 'health_serv',
    'Heavy and Special Industrial' : 'heavy_ind',
    'Light and General Industrial' : 'light_ind',
    'Office' : 'office',
    'Other' : 'other',
    'Rural' : 'rural',
    'Bulky Goods' : 'bulk_goods',
    'Food and Drink' : 'food_drink',
    'Indoor Entertainment and Recreation' : 'indoor_ent',
    'Indoor Recreation (Court Areas)' : 'indoor_rec',
    'Retail and Services' : 'retail'
}

# Add devgroup count and consumption columns
for k, v in dev_cols.items():
    parcels_devuse[f'{v}_ct'] = 0.0
    parcels_devuse[f'{v}_wc'] = 0.0


# Inspect
parcels_devuse.head()

C:\Users\Daniel Smith\AppData\Local\ESRI\conda\envs\arcgis-dev\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,landno,segpar,propno,prop_use,zone,consumption,gfa_est,prop_type,tag,res_devgroup,...,bulk_goods_ct,bulk_goods_wc,food_drink_ct,food_drink_wc,indoor_ent_ct,indoor_ent_wc,indoor_rec_ct,indoor_rec_wc,retail_ct,retail_wc
0,146617,28663092,53815,RES001,LDR,236.0,326.72,Dwelling house (Approved),Residential,Detached Dwelling,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,996564,28460061,416730,RES014,LDR,0.0,0.02,Body Corporate (Residential),Common,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,145846,32043083,13401,RES001,CR,82.0,227.08,Dwelling house (Approved),Residential,Detached Dwelling,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,144907,20484023,13844,VAC003,CN,0.0,0.00,Vacant land,No Development,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,112272,20689095,50686,RES001,LDR,111.0,306.54,Dwelling house (Approved),Residential,Detached Dwelling,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
''' 
----------------------------------
Run residential calculations per land parcel
----------------------------------
'''

# Select only those non-residential uses
parcels_devuse_res = parcels_devuse.loc[
    (parcels_devuse['tag'] == 'Residential') | \
    (parcels_devuse['tag'] == 'Split') \
].copy()

# Calc by measure and count
for index, row in parcels_devuse_res.iterrows():

    gfa = row.gfa_est * 0.7 # 30% reduction in GFA estimate to account for non-contributory areas (walls & circulation)
    
    if row.measure == 'Unit':

        # Set count
        parcels_devuse_res.at[index, f'{dev_cols[row.res_devgroup]}_ct'] = math.ceil(gfa / row.unit_size_sqm)

        # Set consumption
        parcels_devuse_res.at[index, f'{dev_cols[row.res_devgroup]}_wc'] = row.consumption

    elif row.measure == 'Count':

        # Set count
        parcels_devuse_res.at[index, f'{dev_cols[row.res_devgroup]}_ct'] = row.unit_count

        # Set consumption
        parcels_devuse_res.at[index, f'{dev_cols[row.res_devgroup]}_wc'] = row.consumption

    elif row.measure == 'GFA':

        # Set count
        parcels_devuse_res.at[index, f'{dev_cols[row.res_devgroup]}_ct'] = gfa

        # Set consumption
        parcels_devuse_res.at[index, f'{dev_cols[row.res_devgroup]}_wc'] = row.consumption

    else: # Measure = None

        # Set consumption
        parcels_devuse_res.at[index, f'{dev_cols[row.res_devgroup]}_wc'] = row.consumption

    # If the tag is for a split use remove the water consumption details (this will be all assigned to the non-residential component)
    if row.tag == 'Split':

        # Set consumption to 0.0
        parcels_devuse_res.at[index, f'{dev_cols[row.res_devgroup]}_wc'] = 0.0

# Drop redundant columns
parcels_devuse_res = parcels_devuse_res.drop([
    'consumption', 
    'prop_type', 
    'tag', 
    'gfa_est', 
    'zone', 
    'res_devgroup', 
    'measure', 
    'unit_count', 
    'unit_size_sqm', 
    'nonres_devgroup', 
    'nonres_devgroup_alt'
], axis=1)
        
# Inspect
parcels_devuse_res.head()

,landno,segpar,propno,prop_use,det_dwl_ct,det_dwl_wc,att_dwl_ct,att_dwl_wc,accom_long_ct,accom_long_wc,...,bulk_goods_ct,bulk_goods_wc,food_drink_ct,food_drink_wc,indoor_ent_ct,indoor_ent_wc,indoor_rec_ct,indoor_rec_wc,retail_ct,retail_wc
0,146617,28663092,53815,RES001,1.0,236.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,145846,32043083,13401,RES001,1.0,82.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,112272,20689095,50686,RES001,1.0,111.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,330770,63124016,268550,RES001,1.0,161.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,143890,20889026,14991,RES001,1.0,69.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
''' 
----------------------------------
Run non-residential calculations per land parcel
----------------------------------
'''

# Select only those non-residential uses
parcels_devuse_nonres = parcels_devuse.loc[
    (parcels_devuse['tag'] == 'Non-Residential') | \
    (parcels_devuse['tag'] == 'Allocate even') | \
    (parcels_devuse['tag'] == 'Allocate by zone') | \
    (parcels_devuse['tag'] == 'Split') \
].copy()

# Allocate gfa by tag and devtype
for index, row in parcels_devuse_nonres.iterrows():

    gfa = row.gfa_est * 0.7 # 30% reduction in GFA estimate to account for non-contributory areas (walls & circulation)
    
    if row.tag == 'Allocate even':
        
        # Set count
        parcels_devuse_nonres.at[index, f'{dev_cols[row.nonres_devgroup]}_ct'] = gfa * 0.5
        parcels_devuse_nonres.at[index, f'{dev_cols[row.nonres_devgroup_alt]}_ct'] = gfa * 0.5

        # Set consumption
        parcels_devuse_nonres.at[index, f'{dev_cols[row.nonres_devgroup]}_wc'] = row.consumption * 0.5
        parcels_devuse_nonres.at[index, f'{dev_cols[row.nonres_devgroup_alt]}_wc'] = row.consumption * 0.5

    if row.tag == 'Allocate by zone':

        if len([zone for zone in str(row.zone).split(',') if zone in ['LI','MI','WMI']]):

            # Set count
            parcels_devuse_nonres.at[index, f'{dev_cols[row.nonres_devgroup]}_ct'] = gfa

            # Set consumption
            parcels_devuse_nonres.at[index, f'{dev_cols[row.nonres_devgroup]}_wc'] = row.consumption

        else:

            # Set count
            parcels_devuse_nonres.at[index, f'{dev_cols[row.nonres_devgroup_alt]}_ct'] = gfa

            # Set consumption
            parcels_devuse_nonres.at[index, f'{dev_cols[row.nonres_devgroup_alt]}_wc'] = row.consumption

    else:

        # Set count
        parcels_devuse_nonres.at[index, f'{dev_cols[row.nonres_devgroup]}_ct'] = gfa

        # Set consumption
        parcels_devuse_nonres.at[index, f'{dev_cols[row.nonres_devgroup]}_wc'] = row.consumption

# Drop rows
parcels_devuse_nonres = parcels_devuse_nonres.drop([
    'consumption', 
    'prop_type', 
    'gfa_est', 
    'zone', 
    'tag', 
    'nonres_devgroup', 
    'nonres_devgroup_alt', 
    'res_devgroup', 
    'measure', 
    'unit_count', 
    'unit_size_sqm'
], axis=1)
        
# Inspect
parcels_devuse_nonres.head()

,landno,segpar,propno,prop_use,det_dwl_ct,det_dwl_wc,att_dwl_ct,att_dwl_wc,accom_long_ct,accom_long_wc,...,bulk_goods_ct,bulk_goods_wc,food_drink_ct,food_drink_wc,indoor_ent_ct,indoor_ent_wc,indoor_rec_ct,indoor_rec_wc,retail_ct,retail_wc
56,302373,63023029,202579,INF013,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
213,345405,30549362,300920,COM045,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
229,337030,20567564,282210,COM045,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
255,200623,28634045,41035,COM010,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
288,115261,30513029,33574,COM009,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
''' 
----------------------------------
Handle water consumption for common property
----------------------------------
'''

# Select non development uses
parcels_devuse_nodev = parcels_devuse.loc[
    (parcels_devuse['tag'] == 'Common') | \
    (parcels_devuse['tag'] == 'No Development') \
].copy()

# Add common property and non development consumption fields
parcels_devuse_nodev['cmn_res_wc'] = 0.0
parcels_devuse_nodev['cmn_nonres_wc'] = 0.0
parcels_devuse_nodev['nodev_wc'] = 0.0

# Allocate gfa by tag and devtype
for index, row in parcels_devuse_nodev.iterrows():
    
    if row.tag == 'Common':

        if row.prop_use in ['INF021', 'RES014']:
            
            # Set consumption for all common residential to attached dwellings
            parcels_devuse_nodev.at[index, 'cmn_res_wc'] = row.consumption

        else:

            # Set consumption for common non-residential to new field
            parcels_devuse_nodev.at[index, 'cmn_nonres_wc'] = row.consumption

    else:

        # Set consumption for no-development types
        parcels_devuse_nodev.at[index, 'nodev_wc'] = row.consumption

# Drop redundant columns
parcels_devuse_nodev = parcels_devuse_nodev.drop([
    'consumption',
    'tag', 
    'gfa_est', 
    'zone', 
    'prop_type', 
    'nonres_devgroup', 
    'nonres_devgroup_alt', 
    'res_devgroup', 
    'measure', 
    'unit_count', 
    'unit_size_sqm'
], axis=1)

# Inspect
parcels_devuse_nodev.sort_values(by='cmn_res_wc').tail()

,landno,segpar,propno,prop_use,det_dwl_ct,det_dwl_wc,att_dwl_ct,att_dwl_wc,accom_long_ct,accom_long_wc,...,food_drink_wc,indoor_ent_ct,indoor_ent_wc,indoor_rec_ct,indoor_rec_wc,retail_ct,retail_wc,cmn_res_wc,cmn_nonres_wc,nodev_wc
63727,991825,63215297,367000,INF021,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,128628.0,0.0,0.0
16379,991825,63215130,367000,INF021,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,128628.0,0.0,0.0
79496,991825,63215116,367000,INF021,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,128628.0,0.0,0.0
54640,991825,63215299,367000,INF021,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,128628.0,0.0,0.0
61250,991825,63215229,367000,INF021,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,128628.0,0.0,0.0


In [11]:
''' 
----------------------------------
Combine all land parcels into single dataframe
----------------------------------
'''

# Merge residential, non-reseidential and no development tables
parcels_dev = pd.concat([parcels_devuse_res, parcels_devuse_nonres, parcels_devuse_nodev])

# Fill NaN values on the nodev_wc field
parcels_dev['cmn_res_wc'] = parcels_dev['cmn_res_wc'].fillna(0.0)
parcels_dev['cmn_nonres_wc'] = parcels_dev['cmn_nonres_wc'].fillna(0.0)
parcels_dev['nodev_wc'] = parcels_dev['nodev_wc'].fillna(0.0)

# Inspect
parcels_dev.sort_values(by='det_dwl_wc').tail()

,landno,segpar,propno,prop_use,det_dwl_ct,det_dwl_wc,att_dwl_ct,att_dwl_wc,accom_long_ct,accom_long_wc,...,food_drink_wc,indoor_ent_ct,indoor_ent_wc,indoor_rec_ct,indoor_rec_wc,retail_ct,retail_wc,cmn_res_wc,cmn_nonres_wc,nodev_wc
9817,138417,28582011,32927,RES001,1.0,2798.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27321,303278,20598023,203514,RES001,1.0,3460.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
67841,130948,20638046,42989,RES001,1.0,3690.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53177,155898,32935205,22112,RES001,1.0,4803.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
72135,151887,28520018,34489,RES001,1.0,7726.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
''' 
----------------------------------
Split parcels into aggregates and otherwise
----------------------------------
'''

# Import the DCDB aggregates table
aggregates = pd.DataFrame.spatial.from_table(os.path.join(curr_gdb_path, 'GEN_TB_DCDB_Aggregates')).drop(['OBJECTID'], axis=1)

# Join the aggid onto the land parcels
parcels_dev_agg = pd.merge(parcels_dev, aggregates, on='segpar', how='left')

# split the aggregate and non-aggregate parcels
parcels_dev_noagg = parcels_dev_agg.loc[parcels_dev_agg['aggid'].isnull()].drop(['aggid'], axis=1).reset_index(drop=True)
parcels_dev_agg = parcels_dev_agg.loc[parcels_dev_agg['aggid'].notnull()].reset_index(drop=True)

# Inspect
parcels_dev_agg.head()

,landno,segpar,propno,prop_use,det_dwl_ct,det_dwl_wc,att_dwl_ct,att_dwl_wc,accom_long_ct,accom_long_wc,...,indoor_ent_ct,indoor_ent_wc,indoor_rec_ct,indoor_rec_wc,retail_ct,retail_wc,cmn_res_wc,cmn_nonres_wc,nodev_wc,aggid
0,145036,32041024,14893,RES001,1.0,88.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AGG001759
1,117547,20514039,50921,RES001,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AGG000240
2,113842,30591118,43282,RES001,1.0,319.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AGG002829
3,366825,20647074,347550,RES001,1.0,39.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AGG000054
4,360235,20516039,333390,RES001,1.0,65.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AGG000243


In [13]:
''' 
----------------------------------
Handle development calculations for aggregate properties
----------------------------------
'''

# Import the DCDB aggregate details table
agg_details = pd.DataFrame.spatial.from_table(os.path.join(curr_gdb_path, 'GEN_TB_DCDB_AggregateDetails')).drop(['OBJECTID'], axis=1)

# Get all holding aggregates
holdings = agg_details.query("aggtype == 'holding'")

# Get all holding land numbers as set
holding_landnos = {*holdings['landno'].to_list()}

# Select all land parcels that are part of a property holding
parcels_dev_agg_holding = parcels_dev_agg.query("landno in @holding_landnos").drop(['landno', 'propno', 'segpar', 'prop_use'], axis=1)

# For each holding aggregate select the maximum value across all development measures
agg_holdings = parcels_dev_agg_holding.groupby('aggid').agg('max').reset_index()

# Select all other non-holding parcels
parcels_dev_agg_noholding = parcels_dev_agg.query("landno not in @holding_landnos").drop(['propno', 'segpar', 'prop_use'], axis=1)

# Create an empty dictionary to hold the land aggregate operations
land_agg_ops = {}

# Take the max of any counts and take the average water consumption data per land parcel
for col in [x for x in list(parcels_dev_agg_noholding.columns) if x != 'landno']:

    if col.endswith('_ct'):

        land_agg_ops[col] = np.max

    elif col.endswith('_wc'):

        land_agg_ops[col] = np.mean

    else: # This will be 'aggid'

        land_agg_ops[col] = 'count'

# Aggregate by land parcel
agg_other_land = parcels_dev_agg_noholding.groupby('landno').agg(land_agg_ops).reset_index()

# For the land consumption data divide the aggregated total by the number of aggregates to get proper mean per unit
for index, row in agg_other_land.iterrows():
    
    for col, val in row.iteritems():

        if col.endswith('_wc'):

            agg_other_land.at[index, col] = round(val / row.aggid, 2)

# Drop aggid column
agg_other_land = agg_other_land.drop(['aggid'], axis=1)

# Get the aggregate and land numbers of non-holding aggregates
agg_other = parcels_dev_agg_noholding.loc[:, ('aggid', 'landno')].reset_index(drop=True)

# Merge the land counts and consumption records back onto the aggregates
agg_other = pd.merge(agg_other, agg_other_land, on='landno', how='left')

# Sum all development measures then by aggregate id
agg_other = agg_other.groupby('aggid').agg('sum').reset_index()

# For non-holding aggregates all dwellings are measured as attached stock
agg_other['att_dwl_ct'] = agg_other['det_dwl_ct'] + agg_other['att_dwl_ct']
agg_other['det_dwl_ct'] = 0.0

agg_other['att_dwl_wc'] = agg_other['det_dwl_wc'] + agg_other['att_dwl_wc']
agg_other['det_dwl_wc'] = 0.0

# Concatenate all aggregates into single table
agg_all = pd.concat([agg_holdings, agg_other]).sort_values(by='aggid').reset_index(drop=True)

# Inspect
agg_all.head()


,aggid,det_dwl_ct,det_dwl_wc,att_dwl_ct,att_dwl_wc,accom_long_ct,accom_long_wc,accom_short_ct,accom_short_wc,emer_serv_ct,...,food_drink_wc,indoor_ent_ct,indoor_ent_wc,indoor_rec_ct,indoor_rec_wc,retail_ct,retail_wc,cmn_res_wc,cmn_nonres_wc,nodev_wc
0,AGG000001,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,954.00,0.00,0.0
1,AGG000002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,119.01,0.0
2,AGG000003,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1914.00,0.00,0.0
3,AGG000004,0.0,0.0,146.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9566.01,0.00,0.0
4,AGG000005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,540.00,0.0


In [14]:
''' 
----------------------------------
Combine all aggregates into a single dataframe
----------------------------------
'''

# For each aggregate compile the land, dcdb and property use details
agg_details = parcels_dev_agg.loc[:, ('aggid', 'landno', 'propno', 'segpar', 'prop_use')].groupby('aggid').agg(set).reset_index()

# Turn the sets into strings
agg_details['landno'] = agg_details['landno'].apply(lambda x: ';'.join(map(str, x)))
agg_details['propno'] = agg_details['propno'].apply(lambda x: ';'.join(map(str, x)))
agg_details['segpar'] = agg_details['segpar'].apply(lambda x: ';'.join(map(str, x)))
agg_details['prop_use'] = agg_details['prop_use'].apply(lambda x: ';'.join(map(str, x)))

# Merge the property use details back onto the processed aggregates
dev_agg = pd.merge(agg_details, agg_all, on='aggid', how='left')

# Inspect
dev_agg.head()

,aggid,landno,propno,segpar,prop_use,det_dwl_ct,det_dwl_wc,att_dwl_ct,att_dwl_wc,accom_long_ct,...,food_drink_wc,indoor_ent_ct,indoor_ent_wc,indoor_rec_ct,indoor_rec_wc,retail_ct,retail_wc,cmn_res_wc,cmn_nonres_wc,nodev_wc
0,AGG000001,201060;201061;201062;201063;201064;201065;2010...,55008;54999;55000;55001;55002;55003;55004;5500...,30525356;30525357;30525358;30525359;30525360;3...,RES001;INF021,0.0,0.0,9.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,954.00,0.00,0.0
1,AGG000002,201184;201185;201186;201187;201188;201189;2011...,56357;56358;56359;56360;56361;56362;56363;5636...,30514282;30514283;30514284;30514285;30514286;3...,INF020;IND004,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,119.01,0.0
2,AGG000003,201161;201162;201163;201164;201165;201166;2011...,56369;56285;56286;56287;56288;56289;56290;5629...,20327178;20327179;20327180;20327181,RES001;INF021,0.0,0.0,18.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1914.00,0.00,0.0
3,AGG000004,201224;201225;201226;201227;201228;201229;2012...,235330;235270;235280;56726;56727;56728;56729;5...,54498400;54498401;54498403;54498345;54498346;5...,RES001;INF021,0.0,0.0,146.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9566.01,0.00,0.0
4,AGG000005,201349;201350;201351;201352;201353;201354;2013...,57152;57153;57157;57142;57143;57144;57145;5714...,28401048;28401045;28401046;28401047,INF020;IND004,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,540.00,0.0


In [15]:
''' 
----------------------------------
Join the development calculations onto the property base
----------------------------------
'''

# Import the Property Base features
propbase = pd.DataFrame.spatial.from_featureclass(os.path.join(curr_gdb_path, 'GEN_FC_PropertyBase')).drop(['OBJECTID'], axis=1)

# Split the propbase by aggregate
propbase_agg = propbase.query("aggid == aggid").loc[:, ('pbno', 'aggid', 'sa2_name')]
propbase_noagg = propbase.query("aggid != aggid").loc[:, ('pbno', 'segpar', 'sa2_name')]

# Join in the development metrics
propbase_agg = pd.merge(propbase_agg, dev_agg, on='aggid', how='left').dropna()
propbase_noagg = pd.merge(propbase_noagg, parcels_dev_noagg, on='segpar', how='left').dropna()

# Convert required columns to strings
propbase_noagg[['landno', 'propno', 'segpar']] = propbase_noagg[['landno', 'propno', 'segpar']].astype(int)

# Combine aggregate and non aggregate tables
propbase_dev = pd.concat([propbase_agg, propbase_noagg])

# Fill nan
propbase_dev['aggid'] = propbase_dev['aggid'].fillna('')

# Merge duplicate rows taking the maximum value in all cases
propbase_dev = propbase_dev.groupby('pbno').agg('max').reset_index()

# Inspect
propbase_dev.head()

,pbno,aggid,sa2_name,landno,propno,segpar,prop_use,det_dwl_ct,det_dwl_wc,att_dwl_ct,...,food_drink_wc,indoor_ent_ct,indoor_ent_wc,indoor_rec_ct,indoor_rec_wc,retail_ct,retail_wc,cmn_res_wc,cmn_nonres_wc,nodev_wc
0,8000000,AGG000001,Cleveland,201060;201061;201062;201063;201064;201065;2010...,55008;54999;55000;55001;55002;55003;55004;5500...,30525356;30525357;30525358;30525359;30525360;3...,RES001;INF021,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,954.00,0.00,0.0
1,8000001,AGG000002,Cleveland,201184;201185;201186;201187;201188;201189;2011...,56357;56358;56359;56360;56361;56362;56363;5636...,30514282;30514283;30514284;30514285;30514286;3...,INF020;IND004,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,119.01,0.0
2,8000002,AGG000003,Redland Islands,201161;201162;201163;201164;201165;201166;2011...,56369;56285;56286;56287;56288;56289;56290;5629...,20327178;20327179;20327180;20327181,RES001;INF021,0.0,0.0,18.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1914.00,0.00,0.0
3,8000003,AGG000004,Cleveland,201224;201225;201226;201227;201228;201229;2012...,235330;235270;235280;56726;56727;56728;56729;5...,54498400;54498401;54498403;54498345;54498346;5...,RES001;INF021,0.0,0.0,146.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9566.01,0.00,0.0
4,8000004,AGG000005,Capalaba,201349;201350;201351;201352;201353;201354;2013...,57152;57153;57157;57142;57143;57144;57145;5714...,28401048;28401045;28401046;28401047,INF020;IND004,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,540.00,0.0


In [16]:
''' 
----------------------------------
Allocate water usage on common property properly
----------------------------------
'''

# Specify the residential and non residential groups
res_groups = ['att_dwl', 'det_dwl', 'accom_long']
nonres_groups = [ 
    'accom_short',
    'bulk_goods',
    'care_accom',
    'childcare',
    'education',
    'emer_serv',
    'food_drink',
    'health_serv',
    'heavy_ind',
    'hospital',
    'indoor_ent',
    'indoor_rec',
    'light_ind',
    'office',
    'other',
    'pl_of_ass',
    'retail',
    'rural'
]

# Get a copy of the propbase dev df
propbase_dev_wc = propbase_dev.copy()

# Allocate the common property consumption data to the appropriate development groups
for index, row in propbase_dev_wc.iterrows():
    
    # Get the common property consumpton data
    cmn_res = row.cmn_res_wc
    cmn_nonres = row.cmn_nonres_wc   

    # Handle the residential allocation
    if cmn_res:

        # Track the total residential units and remaining water consumption
        units = 0.0
        remainder = cmn_res

        # Get the total number of units across all residential development groups
        for col, val in row.iteritems():

            if col[:-3] in res_groups and col.endswith('_ct'):

                units += val

        for col, val in row.iteritems():

            if col[:-3] in res_groups and col.endswith('_ct'):

                if val > 0.0:

                    share = round(cmn_res * round(units / val, 4), 2)

                    if share > row[f'{col[:-3]}_wc']:

                        propbase_dev_wc.at[index, f'{col[:-3]}_wc'] = share
                        remainder -= share

                    else:

                        remainder -= row[f'{col[:-3]}_wc']

                    del(share)

        # Record the unallocated water consumption
        propbase_dev_wc.at[index, 'cmn_res_wc'] = round(remainder, 2)

        del(remainder)

     # Handle the allocation from non-residential common property
    if cmn_nonres:

        # Track the total GFA and remaining water consumption
        gfa = 0.0
        remainder = cmn_nonres

        # Get the total GFA across all non-residential development groups
        for col, val in row.iteritems():

            if col[:-3] in nonres_groups and col.endswith('_ct'):

                gfa += val

        # Allocate the common property water usage based on the amount each
        # groups GFA factors into the total
        for col, val in row.iteritems():

            if col[:-3] in nonres_groups and col.endswith('_ct'):

                if val > 0.0:

                    share = round(cmn_nonres * round(units / val, 4), 2)

                    if share > row[f'{col[:-3]}_wc']:

                        propbase_dev_wc.at[index, f'{col[:-3]}_wc'] = share
                        remainder -= share

                    else:

                        remainder -= row[f'{col[:-3]}_wc']

                    del(share)

        # Record the unallocated water consumption
        propbase_dev_wc.at[index, 'cmn_nonres_wc'] = round(remainder,  2)

        del(remainder)

    del(cmn_res, cmn_nonres)

# Inspect
propbase_dev_wc.head()

,pbno,aggid,sa2_name,landno,propno,segpar,prop_use,det_dwl_ct,det_dwl_wc,att_dwl_ct,...,food_drink_wc,indoor_ent_ct,indoor_ent_wc,indoor_rec_ct,indoor_rec_wc,retail_ct,retail_wc,cmn_res_wc,cmn_nonres_wc,nodev_wc
0,8000000,AGG000001,Cleveland,201060;201061;201062;201063;201064;201065;2010...,55008;54999;55000;55001;55002;55003;55004;5500...,30525356;30525357;30525358;30525359;30525360;3...,RES001;INF021,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
1,8000001,AGG000002,Cleveland,201184;201185;201186;201187;201188;201189;2011...,56357;56358;56359;56360;56361;56362;56363;5636...,30514282;30514283;30514284;30514285;30514286;3...,INF020;IND004,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,118.70,0.0
2,8000002,AGG000003,Redland Islands,201161;201162;201163;201164;201165;201166;2011...,56369;56285;56286;56287;56288;56289;56290;5629...,20327178;20327179;20327180;20327181,RES001;INF021,0.0,0.0,18.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
3,8000003,AGG000004,Cleveland,201224;201225;201226;201227;201228;201229;2012...,235330;235270;235280;56726;56727;56728;56729;5...,54498400;54498401;54498403;54498345;54498346;5...,RES001;INF021,0.0,0.0,146.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
4,8000004,AGG000005,Capalaba,201349;201350;201351;201352;201353;201354;2013...,57152;57153;57157;57142;57143;57144;57145;5714...,28401048;28401045;28401046;28401047,INF020;IND004,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,441.88,0.0


In [17]:
''' 
----------------------------------
Get the total consumption rates by development group
----------------------------------
'''
# Drop the first seven columns of the property base layer
total_cons = propbase_dev_wc.iloc[:, 7:].transpose().sum(axis=1)

# Inspect
total_cons[:4]

det_dwl_ct      53128.00
det_dwl_wc    9309954.00
att_dwl_ct       9657.00
att_dwl_wc    1538849.25
dtype: float64

In [18]:
''' 
----------------------------------
Restructure consumption by development category
----------------------------------
'''

df_cols = ['id', 'class', 'category', 'group', 'dev_units', 'consumption']
df = pd.DataFrame( columns=df_cols)

group_details = {
    'att_dwl' : ['RE-DW-MD', 'Residential', 'Attached Dwelling', 'Multiple Dwelling'], 
    'det_dwl': ['RE-DW-DD', 'Residential', 'Detached Dwelling', 'Detached Dwelling'], 
    'accom_long' : ['RE-AC-LT', 'Residential', 'Attached Dwelling', 'Accommodation - Long-Term'],
    'accom_short': ['NR-AC-ST', 'Non-Residential', 'Other', 'Accommodation - Short-Term'],
    'bulk_goods': ['NR-RE-BG', 'Non-Residential', 'Retail', 'Bulky Goods'],
    'care_accom': ['NR-HE-CA', 'Non-Residential', 'Health', 'Care Accommodation'],
    'childcare': ['NR-ED-CH', 'Non-Residential', 'Education', 'Childcare'],
    'education': ['NR-ED-ED', 'Non-Residential', 'Education', 'Education'],
    'emer_serv': ['NR-CO-ES', 'Non-Residential', 'Community', 'Emergency Services'],
    'food_drink': ['NR-RE-FD', 'Non-Residential', 'Retail', 'Food and Drink'],
    'health_serv': ['NR-HE-HS', 'Non-Residential', 'Health', 'Health Services'],
    'heavy_ind': ['NR-IN-HI', 'Non-Residential', 'Industrial', 'Heavy and Special Industrial'],
    'hospital': ['NR-HE-HO', 'Non-Residential', 'Health', 'Hospital'],
    'indoor_ent': ['NR-RE-IE', 'Non-Residential', 'Retail', 'Indoor Entertainment and Recreation'],
    'indoor_rec': ['NR-RE-IR', 'Non-Residential', 'Retail', 'Indoor Recreation (Court Areas)'],
    'light_ind': ['NR-IN-LI', 'Non-Residential', 'Industrial', 'Light and General Industrial'],
    'office': ['NR-OF-OF', 'Non-Residential', 'Commercial', 'Office'],
    'other': ['NR-OT-OT', 'Non-Residential', 'Other', 'Other'],
    'pl_of_ass': ['NR-CO-PA', 'Non-Residential', 'Community', 'Places of Assembly'],
    'retail': ['NR-RE-RS', 'Non-Residential', 'Retail', 'Retail and Services'],
    'rural': ['NR-OT-RU', 'Non-Residential', 'Other', 'Rural'],
    'cmn_res': ['NA-NA-NA', 'Not Applicable', 'Not Applicable', 'Residential Common Property (Unallocated)'],
    'cmn_nonres': ['NA-NA-NA', 'Not Applicable', 'Not Applicable', 'Non-Residential Common Property (Unallocated)'],
    'nodev' : ['NA-NA-NA', 'Not Applicable', 'Not Applicable', 'Non Growth Development']
}

for group, details in group_details.items():

    d = []

    consumption = 0.0
    count = 0.0
    avg_annual = 0.0
    avg_daily = 0.0
    ep = 0.0
    
    consumption = total_cons[f'{group}_wc']

    try:
        count = total_cons[f'{group}_ct']
    except:
        count = 0.0

    d.append(details + [round(count, 2), round(consumption, 2)])

    df = df.append(pd.DataFrame(d, columns=df_cols), ignore_index=True)

cons_calcs = df.sort_values(['class', 'category', 'group']).reset_index(drop=True)

cons_calcs


,id,class,category,group,dev_units,consumption
0,NR-OF-OF,Non-Residential,Commercial,Office,227565.09,73230.29
1,NR-CO-ES,Non-Residential,Community,Emergency Services,8095.25,3550.00
2,NR-CO-PA,Non-Residential,Community,Places of Assembly,84524.72,44161.51
3,NR-ED-CH,Non-Residential,Education,Childcare,24770.84,27373.00
4,NR-ED-ED,Non-Residential,Education,Education,510961.12,179949.00
5,NR-HE-CA,Non-Residential,Health,Care Accommodation,65848.80,80929.00
6,NR-HE-HS,Non-Residential,Health,Health Services,10363.86,7475.35
7,NR-HE-HO,Non-Residential,Health,Hospital,69947.94,56890.00
8,NR-IN-HI,Non-Residential,Industrial,Heavy and Special Industrial,14703.39,3963.00
9,NR-IN-LI,Non-Residential,Industrial,Light and General Industrial,600849.78,503324.38


In [19]:
''' 
----------------------------------
Output conversions table
----------------------------------
'''

outputs_path = os.path.join(data_path, 'idm_outputs')

# Output SA2 metrics to csv
cons_calcs.to_csv(os.path.join(outputs_path, 'OUT_RedlandWater_IDM_WaterConsumption.csv'))

In [20]:
''' 
----------------------------------
Output spatial
----------------------------------
'''

# Set output path
out_gdb_path = os.path.join(spatial_path, 'redland_gam_idm.gdb')

# Join development metrics back onto the probase spatial dataframe
pbout = pd.merge(propbase.loc[:, ('pbno', 'SHAPE')], propbase_dev, on='pbno', how='left').dropna()

# Output probase back into geodatabase
pbout.spatial.to_featureclass(os.path.join(out_gdb_path, 'OUT_FC_PropertyBase_WaterConsumption'), overwrite=True)

'G:\\Shared drives\\PIESolutions_03_Projects\\J000111 - Redlands planning assumption update\\06_Working Documents\\00_GIS Directory\\00_Data\\gam_model_run\\redland_gam_idm.gdb\\OUT_FC_PropertyBase_WaterConsumption'